In [1]:
import os
import json
import collections

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

PATH_BASE = "../input/herbarium-2021-fgvc8/"
PATH_TRAIN = os.path.join(PATH_BASE, "train/")
PATH_TRAIN_META = os.path.join(PATH_TRAIN, "metadata.json")


with open(PATH_TRAIN_META) as json_file:
    metadata = json.load(json_file)

In [2]:
ids = []
categories = []
paths = []

for annotation, image in zip(metadata["annotations"], metadata["images"]):
    assert annotation["image_id"] == image["id"]
    ids.append(image["id"])
    categories.append(annotation["category_id"])
    paths.append(image["file_name"])
        
df_meta = pd.DataFrame({"id": ids, "category": categories, "path": paths})

In [3]:
d_categories = {category["id"]: category["name"] for category in metadata["categories"]}
d_families = {category["id"]: category["family"] for category in metadata["categories"]}
d_orders = {category["id"]: category["order"] for category in metadata["categories"]}

df_meta["category_name"] = df_meta["category"].map(d_categories)
df_meta["family_name"] = df_meta["category"].map(d_families)
df_meta["order_name"] = df_meta["category"].map(d_orders)

df_submission = pd.read_csv(
    "../input/herbarium-2021-fgvc8/sample_submission.csv",
    index_col=0,
)
FULL_PIPELINE = False

In [4]:
import os
import random

import numpy as np
from numpy import save, load
import pandas as pd
import cv2
import albumentations as A
from albumentations import pytorch as ATorch
import torch
from torch.utils import data as torch_data
from torch import nn as torch_nn
from torch.nn import functional as torch_functional
import torchvision
from tqdm import tqdm
from sklearn.metrics.pairwise import euclidean_distances

In [5]:
torch.hub.list('pytorch/vision:v0.6.0')

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip


['alexnet',
 'deeplabv3_resnet101',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'fcn_resnet101',
 'googlenet',
 'inception_v3',
 'mobilenet_v2',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'squeezenet1_0',
 'squeezenet1_1',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn',
 'wide_resnet101_2',
 'wide_resnet50_2']

In [6]:
class ResNet50(torch.nn.Module):
    def __init__(self):
        super().__init__()
        tmp_net = torch.hub.load(
            "pytorch/vision:v0.6.0", "resnet101", pretrained=True
        )
        self.net = torch_nn.Sequential(*(list(tmp_net.children())[:-1]))

    def forward(self, x):
        return self.net(x)
    
class DataRetriever(torch_data.Dataset):
    def __init__(
        self, 
        paths, 
        categories=None,
        transforms=None,
        base_path=PATH_TRAIN
    ):
        self.paths = paths
        self.categories = categories
        self.transforms = transforms
        self.base_path = base_path
          
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        img = cv2.imread(os.path.join(self.base_path, self.paths[index]))
        img = cv2.resize(img, (224, 224))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
        
        if self.categories is None:
            return img
        
        y = self.categories[index] 
        return img, y
    
    
def get_transforms():
    return A.Compose(
        [
            A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                p=1.0
            ),
            ATorch.transforms.ToTensorV2(p=1.0),
        ], 
        p=1.0
    )

df_train = df_meta[["category", "path"]].sort_values(by="category")

In [7]:
tmp_path = df_train["path"].tolist()
tmp_category = df_train["category"].tolist()
# If FULL_PIPELINE is False we use small subset of data
if not FULL_PIPELINE:
    tmp_path = tmp_path[:256 * 8]
    tmp_category = tmp_category[:256 * 8]

train_data_retriever = DataRetriever(
    tmp_path,
    tmp_category,
    transforms=get_transforms(),
)

train_loader = torch_data.DataLoader(
    train_data_retriever,
    batch_size=256,
    shuffle=False,
    num_workers=8,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ResNet50()
model.to(device)
model.eval();

category_counts = collections.Counter(df_train["category"].tolist())

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

In [8]:
final_vectors = np.zeros((len(category_counts), 2048))

with torch.no_grad():
    for batch in tqdm(train_loader):
        X, y = batch
        vectors = model(X.to(device)).mean(axis=(2, 3))
        
        _y = y.numpy().tolist()
        for ind in range(len(_y)):
            final_vectors[_y[ind]] += vectors[ind].cpu().numpy().copy() / category_counts[_y[ind]]

 12%|█▎        | 1/8 [01:14<08:42, 74.59s/it]


KeyboardInterrupt: 

In [ ]:
save("average_vectors.npy", final_vectors)
final_vectors = load("average_vectors.npy")

In [ ]:
PATH_TEST = os.path.join(PATH_BASE, "test/")
PATH_TEST_META = os.path.join(PATH_TEST, "metadata.json")


with open(PATH_TEST_META) as json_file:
    metadata = json.load(json_file)

    
id2path = {
    img["id"]: img["file_name"] for img in metadata["images"]
}
df_submission = pd.read_csv(
    "../input/herbarium-2021-fgvc8/sample_submission.csv",
    index_col=0,
)

df_submission["Id"] = df_submission.index
df_submission["Path"] = df_submission["Id"].map(lambda x: id2path[x])

In [ ]:
tmp_path = df_submission["Path"].tolist()
# If FULL_PIPELINE is False we use small subset of data
if not FULL_PIPELINE:
    tmp_path = tmp_path[:256 * 2]

test_data_retriever = DataRetriever(
    tmp_path,
    transforms=get_transforms(),
    base_path=PATH_TEST,
)

test_loader = torch_data.DataLoader(
    test_data_retriever,
    batch_size=256,
    shuffle=False,
    num_workers=8,
)

In [ ]:
res = []

with torch.no_grad():
    for ind, X in enumerate(tqdm(test_loader)):
        vectors = model(X.to(device)).mean(axis=(2, 3))
        tmp = euclidean_distances(vectors.cpu().numpy(), final_vectors)
        res.extend(list(tmp.argmin(axis=1)))

In [ ]:
df_submission.iloc[:len(res), 0] = res

df_submission[["Predicted"]].to_csv("submission.csv")

pd.read_csv("submission.csv", index_col=0)